In [6]:
import torch 
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import h5py
import sys 
models_folder = "C:\\Users\\noedi\\Desktop\\Stage été 2022\\Code stage\\Vae Galaxies\\vae-project\\Models architecture"
sys.path.append(models_folder)

In [2]:
device = 'cuda' if torch.cuda.is_available else 'cpu'

## Importing the model and loading the trained weights

In [10]:
weights_path =  'C:\\Users\\noedi\\Desktop\\Stage été 2022\\Code stage\\Vae Galaxies\\Data\\weights + losses\\galaxy_w_redshift\\cvae\\z32_beta0.1_weights.pt'
weights, hyperparameters = torch.load(weights_path)


z_dim = hyperparameters["z_dim"]
batchsize = hyperparameters["batch size"]


In [11]:
from cvae2 import VariationalAutoencoder
nc, nf =1, 64
vae = VariationalAutoencoder(nc, nf, z_dim).to(device)

## Importing data 

In [ ]:
class dataset(torch.utils.data.Dataset):
    def __init__(self, file):
        self.file = file
        self.hdf = h5py.File(file, 'r')
        self.datasets = list(self.hdf.keys())

    def __len__(self):
        return len(self.hdf[self.datasets[0]])
    
    def __getitem__(self,idx):
        
        x = self.hdf[self.datasets[0]][idx]
        z = self.hdf[self.datasets[1]][idx]

        return x, z

In [12]:
data_cosmic_survey = 'C:\\Users\\noedi\\Desktop\\Stage été 2022\\Code stage\\Vae Galaxies\\Data\\cosmos_25.2_all_with_zphot.h5'
galaxy_dataset = dataset(data_cosmic_survey)

In [26]:
mu = torch.empty(size = (len(galaxy_dataset), z_dim))

for i, x in enumerate(galaxy_dataset):
     galaxy_image = torch.from_numpy(x[0]).unsqueeze(0).to(device)
     condition = torch.tensor([x[1]]).to(device)
     z = vae.encoder(galaxy_image, condition)
     mu[i] = vae.encoder.mu

RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 4.00 GiB total capacity; 3.43 GiB already allocated; 0 bytes free; 3.48 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
for i in range(z_dim): 
    var = "mu" + f"{i}"
    locals()[var] = mu[:,i]